## Context

Banks incur significant losses due to default in loans. This has led to a tightening up of loan underwriting and has increased loan rejection rates. The need for a better credit risk scoring model is also raised by banks.

The CNK bank has collected customer data for the past few years and wants to build a model to predict if a customer coming to purchase a loan is a good customer (will not default) or a bad customer (will default).

In [84]:
# To help with reading and manipulation of data
import numpy as np
import pandas as pd

# To help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# To split the data
from sklearn.model_selection import train_test_split

# To impute missing values
from sklearn.impute import SimpleImputer

# To build a Random forest classifier
from sklearn.ensemble import RandomForestClassifier

# Importing for feature engineering and modeling
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer

# To tune a model
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# To get different performance metrics
import sklearn.metrics as metrics
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    recall_score,
    accuracy_score,
    precision_score,
    f1_score,
)

# To suppress warnings
import warnings

warnings.filterwarnings("ignore")

In [85]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
df = pd.read_csv("/content/drive/MyDrive/DSBA/Loan payments data.csv")

In [87]:
data = df.copy()

In [88]:
data.head()

,Loan_ID,loan_status,Principal,terms,effective_date,due_date,paid_off_time,past_due_days,age,education,Gender
0,xqd20166231,PAIDOFF,1000,30,9/8/2016,10/7/2016,9/14/2016 19:31,NaN,45,High School or Below,male
1,xqd20168902,PAIDOFF,1000,30,9/8/2016,10/7/2016,10/7/2016 9:00,NaN,50,Bechalor,female
2,xqd20160003,PAIDOFF,1000,30,9/8/2016,10/7/2016,9/25/2016 16:58,NaN,33,Bechalor,female
3,xqd20160004,PAIDOFF,1000,15,9/8/2016,9/22/2016,9/22/2016 20:00,NaN,27,college,male
4,xqd20160005,PAIDOFF,1000,30,9/9/2016,10/8/2016,9/23/2016 21:36,NaN,28,college,female


In [89]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Loan_ID         500 non-null    object 
 1   loan_status     500 non-null    object 
 2   Principal       500 non-null    int64  
 3   terms           500 non-null    int64  
 4   effective_date  500 non-null    object 
 5   due_date        500 non-null    object 
 6   paid_off_time   400 non-null    object 
 7   past_due_days   200 non-null    float64
 8   age             500 non-null    int64  
 9   education       500 non-null    object 
 10  Gender          500 non-null    object 
dtypes: float64(1), int64(3), object(7)
memory usage: 43.1+ KB


In [90]:
data.drop(["Loan_ID"], axis=1, inplace=True)

In [92]:
data["paid_off_time"] = pd.to_datetime(data["paid_off_time"])
data.head()

data["due_date"] = pd.to_datetime(data["due_date"])

data["effective_date"] = pd.to_datetime(data["effective_date"])

In [93]:
import datetime as dt
data["paid_off_month"] = data["paid_off_time"].dt.month
data["paid_off_hour"] = data["paid_off_time"].dt.hour
data["paid_off_dow"] = data["paid_off_time"].dt.dayofweek

data["due_date_month"] = data["due_date"].dt.month 
data["due_date_dow"] = data["due_date"].dt.dayofweek

data["effective_date_month"] = data["effective_date"].dt.month
data["effective_date_dow"] = data["effective_date"].dt.dayofweek


In [94]:
data.head()

,loan_status,Principal,terms,effective_date,due_date,paid_off_time,past_due_days,age,education,Gender,paid_off_month,paid_off_hour,paid_off_dow,due_date_month,due_date_dow,effective_date_month,effective_date_dow
0,PAIDOFF,1000,30,2016-09-08,2016-10-07,2016-09-14 19:31:00,NaN,45,High School or Below,male,9.0,19.0,2.0,10,4,9,3
1,PAIDOFF,1000,30,2016-09-08,2016-10-07,2016-10-07 09:00:00,NaN,50,Bechalor,female,10.0,9.0,4.0,10,4,9,3
2,PAIDOFF,1000,30,2016-09-08,2016-10-07,2016-09-25 16:58:00,NaN,33,Bechalor,female,9.0,16.0,6.0,10,4,9,3
3,PAIDOFF,1000,15,2016-09-08,2016-09-22,2016-09-22 20:00:00,NaN,27,college,male,9.0,20.0,3.0,9,3,9,3
4,PAIDOFF,1000,30,2016-09-09,2016-10-08,2016-09-23 21:36:00,NaN,28,college,female,9.0,21.0,4.0,10,5,9,4


In [95]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   loan_status           500 non-null    object        
 1   Principal             500 non-null    int64         
 2   terms                 500 non-null    int64         
 3   effective_date        500 non-null    datetime64[ns]
 4   due_date              500 non-null    datetime64[ns]
 5   paid_off_time         400 non-null    datetime64[ns]
 6   past_due_days         200 non-null    float64       
 7   age                   500 non-null    int64         
 8   education             500 non-null    object        
 9   Gender                500 non-null    object        
 10  paid_off_month        400 non-null    float64       
 11  paid_off_hour         400 non-null    float64       
 12  paid_off_dow          400 non-null    float64       
 13  due_date_month      

In [96]:
data.drop(["effective_date", "due_date", "paid_off_time"], axis=1, inplace=True)

In [97]:
data["education"].replace("Bechalor", "Bachelor")

0      High School or Below
1                  Bachelor
2                  Bachelor
3                   college
4                   college
               ...         
495    High School or Below
496    High School or Below
497                 college
498                 college
499    High School or Below
Name: education, Length: 500, dtype: object

In [98]:
data.dtypes

loan_status              object
Principal                 int64
terms                     int64
past_due_days           float64
age                       int64
education                object
Gender                   object
paid_off_month          float64
paid_off_hour           float64
paid_off_dow            float64
due_date_month            int64
due_date_dow              int64
effective_date_month      int64
effective_date_dow        int64
dtype: object

In [99]:
data["loan_status"] = data["loan_status"].astype("category")
data["education"] = data["education"].astype("category")
data["Gender"] = data["Gender"].astype("category")

In [100]:
data.isnull().sum()

loan_status               0
Principal                 0
terms                     0
past_due_days           300
age                       0
education                 0
Gender                    0
paid_off_month          100
paid_off_hour           100
paid_off_dow            100
due_date_month            0
due_date_dow              0
effective_date_month      0
effective_date_dow        0
dtype: int64

In [101]:
data["loan_status"].value_counts(1)

PAIDOFF               0.6
COLLECTION            0.2
COLLECTION_PAIDOFF    0.2
Name: loan_status, dtype: float64

In [102]:
data["loan_status"] = data["loan_status"].map({"PAIDOFF":0, "COLLECTION":1, "COLLECTION_PAIDOFF":1})

In [103]:
data.iloc[:,[3,7,8,9]]

,past_due_days,paid_off_month,paid_off_hour,paid_off_dow
0,NaN,9.0,19.0,2.0
1,NaN,10.0,9.0,4.0
2,NaN,9.0,16.0,6.0
3,NaN,9.0,20.0,3.0
4,NaN,9.0,21.0,4.0
...,...,...,...,...
495,3.0,10.0,19.0,4.0
496,14.0,10.0,20.0,0.0
497,3.0,9.0,11.0,3.0
498,1.0,11.0,22.0,4.0


In [104]:
data.head()

,loan_status,Principal,terms,past_due_days,age,education,Gender,paid_off_month,paid_off_hour,paid_off_dow,due_date_month,due_date_dow,effective_date_month,effective_date_dow
0,0,1000,30,NaN,45,High School or Below,male,9.0,19.0,2.0,10,4,9,3
1,0,1000,30,NaN,50,Bechalor,female,10.0,9.0,4.0,10,4,9,3
2,0,1000,30,NaN,33,Bechalor,female,9.0,16.0,6.0,10,4,9,3
3,0,1000,15,NaN,27,college,male,9.0,20.0,3.0,9,3,9,3
4,0,1000,30,NaN,28,college,female,9.0,21.0,4.0,10,5,9,4


In [105]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   loan_status           500 non-null    int64   
 1   Principal             500 non-null    int64   
 2   terms                 500 non-null    int64   
 3   past_due_days         200 non-null    float64 
 4   age                   500 non-null    int64   
 5   education             500 non-null    category
 6   Gender                500 non-null    category
 7   paid_off_month        400 non-null    float64 
 8   paid_off_hour         400 non-null    float64 
 9   paid_off_dow          400 non-null    float64 
 10  due_date_month        500 non-null    int64   
 11  due_date_dow          500 non-null    int64   
 12  effective_date_month  500 non-null    int64   
 13  effective_date_dow    500 non-null    int64   
dtypes: category(2), float64(4), int64(8)
memory usage: 48.3 KB

In [106]:
X = data.drop(["loan_status"], axis=1)
y = data["loan_status"]

X = pd.get_dummies(X, drop_first=True)

In [107]:
#Splitting the data into training, validation and test set

#First we split the data into 2 parts, temporary and test set
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=5, stratify=y
)

#then we split the temporary dataset into train and validation
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=5, stratify=y_temp
)

print(X_train.shape, X_val.shape, X_test.shape)

(320, 15) (80, 15) (100, 15)


In [108]:
# Let's impute the missing values
imp_median = SimpleImputer(missing_values=np.nan, strategy="median")

# fit the imputer on train data and transform the train data
X_train["past_due_days"] = imp_median.fit_transform(X_train[["past_due_days"]])

# transform the validation and test data using the imputer fit on train data
X_val["past_due_days"] = imp_median.transform(X_val[["past_due_days"]])
X_test["past_due_days"] = imp_median.transform(X_test[["past_due_days"]])

X_train["paid_off_month"] = imp_median.fit_transform(X_train[["paid_off_month"]])

# transform the validation and test data using the imputer fit on train data
X_val["paid_off_month"] = imp_median.transform(X_val[["paid_off_month"]])
X_test["paid_off_month"] = imp_median.transform(X_test[["paid_off_month"]])

X_train["paid_off_hour"] = imp_median.fit_transform(X_train[["paid_off_hour"]])

# transform the validation and test data using the imputer fit on train data
X_val["paid_off_hour"] = imp_median.transform(X_val[["paid_off_hour"]])
X_test["paid_off_hour"] = imp_median.transform(X_test[["paid_off_hour"]])

X_train["paid_off_dow"] = imp_median.fit_transform(X_train[["paid_off_dow"]])

# transform the validation and test data using the imputer fit on train data
X_val["paid_off_dow"] = imp_median.transform(X_val[["paid_off_dow"]])
X_test["paid_off_dow"] = imp_median.transform(X_test[["paid_off_dow"]])

In [79]:
#transformer = ColumnTransformer(transformers=[('num', SimpleImputer(strategy='median'), [3,7,8,9])], remainder='passthrough')

#X_train = transformer.fit_transform(X_train)
#X_val = transformer.fit_transform(X_val)
#X_test = transformer.fit_transform(X_test)

In [109]:
print("Target value ratio in y")
print(y.value_counts(1))
print("*" * 80)
print("Target value ratio in y_train")
print(y_train.value_counts(1))
print("*" * 80)
print("Target value ratio in y_val")
print(y_val.value_counts(1))
print("*" * 80)
print("Target value ratio in y_test")
print(y_test.value_counts(1))
print("*" * 80)

Target value ratio in y
0    0.6
1    0.4
Name: loan_status, dtype: float64
********************************************************************************
Target value ratio in y_train
0    0.6
1    0.4
Name: loan_status, dtype: float64
********************************************************************************
Target value ratio in y_val
0    0.6
1    0.4
Name: loan_status, dtype: float64
********************************************************************************
Target value ratio in y_test
0    0.6
1    0.4
Name: loan_status, dtype: float64
********************************************************************************


## Model evaluation criterion


**What does a bank want?**
* A bank wants to minimize the loss - it can face 2 types of losses here: 
   * Whenever a bank lends money to a customer, they don't return it.
   * A bank doesn't lend money to a customer thinking a customer will default but in reality, the customer won't - opportunity loss.

**Which loss is greater ?**
* Lending to a customer who wouldn't be able to pay back.

**Since we want to reduce loan defaults we should use Recall as a metric of model evaluation instead of accuracy.**

* Recall - It gives the ratio of True positives to Actual positives, so high Recall implies low false negatives, i.e. low chances of predicting a bad customer as a good customer.


# Hyperparameter Tuning

In [110]:
# model without hyperparameter tuning
rf = RandomForestClassifier(random_state=1)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [111]:
# Checking recall score on train and validation set
print("Recall on train and validation set")
print(recall_score(y_train, rf.predict(X_train)))
print(recall_score(y_val, rf.predict(X_val)))
print("")

# Checking Precision score on train and validation set
print("Precision on train and validation set")
print(precision_score(y_train, rf.predict(X_train)))
print(precision_score(y_val, rf.predict(X_val)))

print("")

# Checking Accuracy score on train and validation set
print("Accuracy on train and validation set")
print(accuracy_score(y_train, rf.predict(X_train)))
print(accuracy_score(y_val, rf.predict(X_val)))

Recall on train and validation set
1.0
1.0

Precision on train and validation set
1.0
1.0

Accuracy on train and validation set
1.0
1.0


## Grid Search CV
* Hyperparameter tuning is also tricky in the sense that there is no direct way to calculate how a change in the hyperparameter value will reduce the loss of your model, so we usually resort to experimentation. i.e we'll use Grid search
* Grid search is a tuning technique that attempts to compute the optimum values of hyperparameters. 
* It is an exhaustive search that is performed on the specific parameter values of a model.
* The parameters of the estimator/model used to apply these methods are optimized by cross-validated grid-search over a parameter grid.

In [112]:
RandomForestClassifier().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [113]:
print(np.arange(0.2, 0.7, 0.1))

print(np.arange(5,10))

[0.2 0.3 0.4 0.5 0.6]
[5 6 7 8 9]


### Let's tune Random forest using Grid Search

In [114]:
%%time

# Choose the type of classifier. 
rf1 = RandomForestClassifier(random_state=1)

# Grid of parameters to choose from
parameters = {"n_estimators": [150,200,250],
    "min_samples_leaf": np.arange(5, 10),
    "max_features": np.arange(0.2, 0.7, 0.1),
    "max_samples": np.arange(0.3, 0.7, 0.1),
    "class_weight" : ['balanced', 'balanced_subsample'],
    "max_depth":np.arange(3,4,5),
    "min_impurity_decrease":[0.001, 0.002, 0.003]
             }

# Type of scoring used to compare parameter combinations
acc_scorer = metrics.make_scorer(metrics.recall_score)

# Run the grid search
grid_obj = GridSearchCV(rf1, parameters, scoring=acc_scorer, cv=5, n_jobs= -1, verbose = 2)
# verbose = 2 tells about the number of fits, which can give an idea of how long will the model take in tuning
# n_jobs = -1 so that all CPU cores can be run parallelly to optimize the Search

grid_obj = grid_obj.fit(X_train, y_train)

# Print the best combination of parameters
grid_obj.best_params_


Fitting 5 folds for each of 1800 candidates, totalling 9000 fits
CPU times: user 36.1 s, sys: 2.83 s, total: 38.9 s
Wall time: 38min 28s


{'class_weight': 'balanced',
 'max_depth': 3,
 'max_features': 0.4000000000000001,
 'max_samples': 0.6000000000000001,
 'min_impurity_decrease': 0.001,
 'min_samples_leaf': 5,
 'n_estimators': 150}

In [115]:
grid_obj.best_score_

0.9763076923076923

In [116]:
# Set the clf to the best combination of parameters
rf1_tuned = RandomForestClassifier(
    class_weight="balanced",
    max_features=0.2,
    max_samples=0.6000000000000001,
    min_samples_leaf=5,
    n_estimators=150,
    max_depth=3,
    random_state=1,
    min_impurity_decrease=0.001,
)

# Fit the best algorithm to the data.
rf1_tuned.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=3, max_features=0.2,
                       max_samples=0.6000000000000001,
                       min_impurity_decrease=0.001, min_samples_leaf=5,
                       n_estimators=150, random_state=1)

In [117]:
# Checking recall score on train and validation set
print("Recall on train and validation set")
print(recall_score(y_train, rf1_tuned.predict(X_train)))
print(recall_score(y_val, rf1_tuned.predict(X_val)))
print("")

# Checking precision score on train and validation set
print("Precision on train and validation set")
print(precision_score(y_train, rf1_tuned.predict(X_train)))
print(precision_score(y_val, rf1_tuned.predict(X_val)))
print("")

# Checking accuracy score on train and validation set
print("Accuracy on train and validation set")
print(accuracy_score(y_train, rf1_tuned.predict(X_train)))
print(accuracy_score(y_val, rf1_tuned.predict(X_val)))

Recall on train and validation set
0.984375
0.96875

Precision on train and validation set
0.9767441860465116
0.96875

Accuracy on train and validation set
0.984375
0.975


## Randomized Search CV
* Random search is a tuning technique that attempts to compute the optimum values of hyperparameters randomly unlike grid search

In [118]:
%%time

# Choose the type of classifier. 
rf2 = RandomForestClassifier(random_state=1)

# Grid of parameters to choose from
parameters = {"n_estimators": [150,200,250],
    "min_samples_leaf": np.arange(5, 10),
    "max_features": np.arange(0.2, 0.7, 0.1), 
    "max_samples": np.arange(0.3, 0.7, 0.1),
    "max_depth":np.arange(3,4,5),
    "class_weight" : ['balanced', 'balanced_subsample'],
    "min_impurity_decrease":[0.001, 0.002, 0.003]
             }

# Type of scoring used to compare parameter combinations
acc_scorer = metrics.make_scorer(metrics.recall_score)

# Run the random search
grid_obj = RandomizedSearchCV(rf2, parameters,n_iter=30, scoring=acc_scorer,cv=5, random_state = 1, n_jobs = -1, verbose = 2)
# using n_iter = 30, so randomized search will try 30 different combinations of hyperparameters
# by default, n_iter = 10

grid_obj = grid_obj.fit(X_train, y_train)

# Print the best combination of parameters
grid_obj.best_params_


Fitting 5 folds for each of 30 candidates, totalling 150 fits
CPU times: user 836 ms, sys: 80.6 ms, total: 916 ms
Wall time: 38 s


{'n_estimators': 150,
 'min_samples_leaf': 5,
 'min_impurity_decrease': 0.002,
 'max_samples': 0.6000000000000001,
 'max_features': 0.4000000000000001,
 'max_depth': 3,
 'class_weight': 'balanced'}

In [119]:
grid_obj.best_score_

0.9763076923076923

In [120]:
# Set the clf to the best combination of parameters
rf2_tuned = RandomForestClassifier(
    class_weight="balanced",
    max_features=0.2,
    max_samples=0.5,
    min_samples_leaf=5,
    n_estimators=150,
    random_state=1,
    max_depth=3,
    min_impurity_decrease=0.003,
)

# Fit the best algorithm to the data.
rf2_tuned.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=3, max_features=0.2,
                       max_samples=0.5, min_impurity_decrease=0.003,
                       min_samples_leaf=5, n_estimators=150, random_state=1)

In [121]:
# Checking recall score on train and validation set
print("Recall on train and validation set")
print(recall_score(y_train, rf2_tuned.predict(X_train)))
print(recall_score(y_val, rf2_tuned.predict(X_val)))
print("")
print("Precision on train and validation set")
# Checking precision score on train and validation set
print(precision_score(y_train, rf2_tuned.predict(X_train)))
print(precision_score(y_val, rf2_tuned.predict(X_val)))
print("")
print("Accuracy on train and validation set")
# Checking accuracy score on train and validation set
print(accuracy_score(y_train, rf2_tuned.predict(X_train)))
print(accuracy_score(y_val, rf2_tuned.predict(X_val)))

Recall on train and validation set
0.9609375
0.96875

Precision on train and validation set
0.9761904761904762
1.0

Accuracy on train and validation set
0.975
0.9875


In [122]:
model = rf1_tuned

In [123]:
# Checking recall score on test set
print("Recall on test set")
print(recall_score(y_test, model.predict(X_test)))
print("")

# Checking precision score on test set
print("Precision on test set")
print(precision_score(y_test, model.predict(X_test)))
print("")

# Checking accuracy score on test set
print("Accuracy on test set")
print(accuracy_score(y_test, model.predict(X_test)))

Recall on test set
1.0

Precision on test set
0.975609756097561

Accuracy on test set
0.99
